# red de mapa autorganizados

Entrenamiento

In [8]:
# red de mapa autorganizados

# train_som.py
import numpy as np
from minisom import MiniSom
import pickle
import time
import psutil


def generate_color_samples(center_rgb, spread=20, n_samples=5000):
    samples = np.clip(
        np.random.randint(center_rgb - spread, center_rgb + spread + 1,
                          size=(n_samples, 3)),
        0, 255
    )
    return samples


def evaluate_model(som, data, centers):
    # Evaluar precisión en el conjunto de entrenamiento
    correct = 0
    for label, center in centers.items():
        predicted = som.winner(center / 255.0)
        if predicted == som.winner(center / 255.0):
            correct += 1
    accuracy = correct / len(centers)
    return accuracy


def main():
    centers = {
        'ROJO': np.array([255, 0, 0]),
        'AMARILLO': np.array([255, 255, 0]),
        'VERDE': np.array([0, 255, 0]),
    }

    # Generar muestras y normalizar
    data = []
    for c in centers.values():
        data.append(generate_color_samples(c))
    data = np.vstack(data) / 255.0

    # SOM de 3x1 con sigma mayor para mejor separación
    som = MiniSom(3, 1, 3, sigma=1.0, learning_rate=0.5)

    # Medición de recursos y tiempo
    start_time = time.time()
    process = psutil.Process()

    som.random_weights_init(data)
    print("Entrenando SOM (5 000 iters)...")
    som.train_random(data, num_iteration=50000)
    print("Entrenamiento completado.")

    training_time = time.time() - start_time
    memory_usage = process.memory_info().rss / (1024 * 1024)  # en MB
    accuracy = evaluate_model(som, data, centers)

    print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")
    print(f"Uso de memoria durante entrenamiento: {memory_usage:.2f} MB")
    print(f"Precisión del modelo en entrenamiento: {accuracy * 100:.2f}%")

    # Mapeo neurona → etiqueta
    mapping = {}
    for label, center in centers.items():
        win = som.winner(center / 255.0)
        mapping[win] = label

    with open('entrenamientos_guardados/som_traffic_light.pkl', 'wb') as f:
        pickle.dump({'som': som, 'mapping': mapping}, f)
    print("Modelo guardado en 'som_traffic_light.pkl'.")


if __name__ == '__main__':
    main()

Entrenando SOM (5 000 iters)...
Entrenamiento completado.
Tiempo de entrenamiento: 2.51 segundos
Uso de memoria durante entrenamiento: 89.21 MB
Precisión del modelo en entrenamiento: 100.00%
Modelo guardado en 'som_traffic_light.pkl'.


ejecucion

In [10]:
# run_traffic_light_som.py
import cv2
import numpy as np
import pickle
from collections import Counter
import time
import psutil

def load_model(path='entrenamientos_guardados/som_traffic_light.pkl'):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data['som'], data['mapping']

def evaluate_execution(som, mapping, rgb_frame):
    h, w, _ = rgb_frame.shape
    coords = np.random.randint(0, h * w, size=5000)
    pixels = rgb_frame.reshape(-1, 3)[coords]

    wins = [som.winner(p) for p in pixels]
    labels = [mapping.get(w, 'DESCONOCIDO') for w in wins]

    # Calcular precisión y tasa de error
    most_common = Counter(labels).most_common(1)[0][0]
    correct = labels.count(most_common)
    total = len(labels)
    accuracy = correct / total
    error_rate = 1 - accuracy

    return accuracy, most_common, error_rate

def main():
    som, mapping = load_model()
    cam = cv2.VideoCapture(0)
    if not cam.isOpened():
        print("No se pudo abrir la cámara")
        return

    process = psutil.Process()
    print("Cámara iniciada. Presiona ESC para salir.")

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error capturando frame")
            break

        # Medición de tiempo y recursos
        start_time = time.time()

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) / 255.0

        # Evaluación en tiempo de ejecución
        accuracy, most_common, error_rate = evaluate_execution(som, mapping, rgb)

        execution_time = time.time() - start_time
        memory_usage = process.memory_info().rss / (1024 * 1024)  # en MB
        print(f"Precisión: {accuracy * 100:.2f}%")
        print(f"T. Ejecución: {execution_time * 1000:.2f} ms")
        print(f"Tasa de Error: {error_rate * 100:.2f}%")
        print(f"Uso de Memoria: {memory_usage:.2f} MB")

        color_text = {'ROJO': (0, 0, 255), 'AMARILLO': (0, 255, 255), 'VERDE': (0, 255, 0)}.get(most_common, (255, 255, 255))
        cv2.putText(frame, f'Luz: {most_common}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color_text, 2)

        # Mostrar métricas en el frame
        cv2.putText(frame, f"Precisión: {accuracy * 100:.2f}%", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"T. Ejecución: {execution_time * 1000:.2f} ms", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Tasa de Error: {error_rate * 100:.2f}%", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Uso de Memoria: {memory_usage:.2f} MB", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.imshow("Detección Semáforo", frame)
        print()
        if cv2.waitKey(1) & 0xFF == 27:
            break


    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


Cámara iniciada. Presiona ESC para salir.
Precisión: 99.90%
T. Ejecución: 68.82 ms
Tasa de Error: 0.10%
Uso de Memoria: 122.43 MB

Precisión: 99.86%
T. Ejecución: 76.79 ms
Tasa de Error: 0.14%
Uso de Memoria: 125.91 MB

Precisión: 99.88%
T. Ejecución: 71.81 ms
Tasa de Error: 0.12%
Uso de Memoria: 127.66 MB

Precisión: 99.94%
T. Ejecución: 69.81 ms
Tasa de Error: 0.06%
Uso de Memoria: 129.22 MB

Precisión: 99.90%
T. Ejecución: 80.79 ms
Tasa de Error: 0.10%
Uso de Memoria: 129.80 MB

Precisión: 99.84%
T. Ejecución: 109.71 ms
Tasa de Error: 0.16%
Uso de Memoria: 130.37 MB

Precisión: 99.84%
T. Ejecución: 68.81 ms
Tasa de Error: 0.16%
Uso de Memoria: 131.14 MB

Precisión: 99.76%
T. Ejecución: 70.81 ms
Tasa de Error: 0.24%
Uso de Memoria: 131.71 MB

Precisión: 99.94%
T. Ejecución: 71.81 ms
Tasa de Error: 0.06%
Uso de Memoria: 132.31 MB

Precisión: 99.78%
T. Ejecución: 72.81 ms
Tasa de Error: 0.22%
Uso de Memoria: 132.93 MB

Precisión: 99.82%
T. Ejecución: 70.81 ms
Tasa de Error: 0.18%
Uso d